## Speed-gradient algorithm for the problem of classifying dynamic objects using artificial neural networks


In [1]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:10000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [2]:
import warnings
warnings.simplefilter('ignore')

import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve, classification_report

%matplotlib inline
#from sklearn.model_selection import train_test_split, GridSearchCV

import torch
from torch import nn, tensor, optim
from torchviz import make_dot

import torch.nn.functional as F

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True


#### Generating data

In [3]:
number_of_objects = 24
number_of_observations = 2
number_of_axes = 2
names_of_axes = ['x', 'y']
obs = ''
axis_index = []

observations = np.array([[1., 1., 2., 2.],
                         [0., 1., 1., 2.],
                         [1., -1., 3., 1.],
                         [3., 0., 1., 3.],
                         [0., -1., -1., -2.],
                         [-1., -1., -3., -3.],
                         [5., 3., 4., 1.],
                         [-1., 1., -2., 0.],
                         [2., 2., 3., 3.],
                         [1., 2., 2., 3.],
                         [3., 1., 5., 3.],
                         [1., 3., -1., 6.],
                         [-1., -2., -2., -3.],
                         [-3., -3., -5., -5.],
                         [4., 1., 3., -1.],
                         [-2., 0., -3., -1.],
                         [2., 2., 2., 3.],
                         [1., 2., 1., 3.],
                         [3., 1., 3., 3.],
                         [1., 3., 2., 3.],
                         [-1., -2., -3., -1.],
                         [-3., -3., -2., -3.],
                         [4., 1., 3., 3.],
                         [-2., 0., -2., -3.]])


target = np.array([1., 1., 1., 1., 1., 1., 1., 1.,
                   1., 1., 1., 1., 1., 1., 1., 1.,
                   -1., -1., -1., -1., -1., -1., -1., -1.])

moment = np.array([0., 0., 0., 0., 0., 0., 0., 0.,
                   1., 1., 1., 1., 1., 1., 1., 1.,
                   1., 1., 1., 1., 1., 1., 1., 1.])

dotX = np.array([[1., 1., 1., 1.],
                 [1., 1., 1., 1.],
                 [2., 2., 2., 2.],
                 [-2., 3., -2., 3.],
                 [-1., -1., -1., -1.],
                 [-2., -2, -2., -2.],
                 [-1., -2., -1., -2.],
                 [-1., -1., -1., -1.],
                 
                 [1., 1., 1., 1.],
                 [1., 1., 1., 1.],
                 [2., 2., 2., 2.],
                 [-2., 3., -2., 3.],
                 [-1., -1., -1., -1.],
                 [-2., -2, -2., -2.],
                 [-1., -2., -1., -2.],
                 [-1., -1., -1., -1.],
                 
                 [1., 1., 0., 1.],
                 [1., 1., 0., 1.],
                 [2., 2., 0., 2.],
                 [-2., 3., 1., 0.],
                 [-1., -1., -2., 1.],
                 [-2., -2, 1., 0.],
                 [-1., -2., -1., 2.],
                 [-1., -1., 0., -3.]])

test = np.array([[3., 3., 4., 4., 2.],
                 [2., 3., 3., 4., 2.],
                 [5., 3., 7., 5., 2.],
                 [-1., 6., -3., 9., 2.],
                 [-2., -3., -3., -4., 2.],
                 [-5., -5., -7., -7., 2.],
                 [3., -1., 2., -3., 2.],
                 [-3., -1., -4., -2., 2.],
                 
                 [3., 3., 3., 4., 2.],
                 [2., 3., 4., 4., 2.],
                 [5., 3., 4., 4., 2.],
                 [-1., 6., 3., 4., 2.],
                 [-2., -3., -4., -2., 2.],
                 [-5., -5., -3., -4., 2.],
                 [3., -1., -3., -4., 2.],
                 [-3., -1., -3., -4., 2.]])

test_target = np.array([1., 1., 1., 1., 1., 1., 1., 1.,
                        -1., -1., -1., -1., -1., -1., -1., -1.])

test_moment = [2, 2, 2, 2, 2, 2, 2, 2,
               2, 2, 2, 2, 2, 2, 2, 2]

observations = observations.T
        
for i in range(1, number_of_observations+1):
    obs += str(i) + ' ' + str(i) + ' '
    for j in names_of_axes:
        axis_index.append(j)
    
observations_index = obs.split()
hier_index = list(zip(observations_index,axis_index))
hier_index = pd.MultiIndex.from_tuples(hier_index)

df = pd.DataFrame(observations, index=hier_index)
df.columns.name = 'object'
df.index.names = ['observation', 'axis']
df = df.T

df['time'] = moment
df['target'] = target
df.to_csv('data.csv', sep='\t', encoding='utf-8')
df.head(5)


observation    1         2      time target
axis           x    y    x    y            
object                                     
0            1.0  1.0  2.0  2.0  0.0    1.0
1            0.0  1.0  1.0  2.0  0.0    1.0
2            1.0 -1.0  3.0  1.0  0.0    1.0
3            3.0  0.0  1.0  3.0  0.0    1.0
4            0.0 -1.0 -1.0 -2.0  0.0    1.0

#### Creating tensors

In [4]:
# N is number of objects; D_in is input dimension; D_out is output dimension.
N, D_in, D_out = df.shape[0], df.shape[1] - 2, 1

time = tensor(df['time'].values, dtype=torch.float32).view(D_out, df.shape[0])
X = tensor(df.iloc[:, :D_in].values, dtype=torch.float32)
target = tensor(df['target'].values, dtype=torch.float32).view(N, D_out)

test_time = tensor(test[:, D_in], dtype=torch.float32).view(D_out, test.shape[0])
test = tensor(test[:, :D_in], dtype=torch.float32)
test_target = tensor(test_target, dtype=torch.float32).view(test_target.shape[0], D_out)

dotX = tensor(dotX, dtype=torch.float32)
dotQmax = tensor(-2.)


#### Models

In [5]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.linear1 = nn.Linear(D_in, D_out)
        self.linear2 = nn.Linear(D_in, D_out)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x, time):
        x1 = self.linear1(x)
        x2 = self.linear2(x * time.T)
        x = x1 + x2
        z_pred = self.sigmoid(x) * 2 - 1
        return z_pred

model = Model()


In [6]:
class ModelBP(nn.Module):
    def __init__(self):
        super(ModelBP, self).__init__()
        self.linear = nn.Linear(D_in, D_out)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.linear(x)
        z_predBP = self.sigmoid(x) * 2 - 1
        return z_predBP

modelBP = ModelBP()


#### Training

In [7]:
#Backpropogation
lossesBP = []
accuracy_historyBP = []
test_lossesBP = []
test_accuracy_historyBP = []

learning_rateBP = 1e-3
loss_fnBP = nn.MSELoss(reduction='sum')
optimizerBP = optim.SGD(modelBP.parameters(), lr=learning_rateBP)

for epoch in range(9000):
    modelBP.train()
    z_predBP = modelBP(X)
    test_predBP = modelBP(test)
    
    lossBP = loss_fnBP(z_predBP, target) / 2
    accuracyBP = (tensor(np.sign(z_predBP.detach().numpy())) == target).float().mean().data.cpu()

    lossBP.backward()
    optimizerBP.step()
    optimizerBP.zero_grad()
    
    modelBP.eval()
    test_lossBP = loss_fnBP(test_predBP, test_target) / 2
    test_accuracyBP = (tensor(np.sign(test_predBP.detach().numpy())) == test_target).float().mean().data.cpu()
    
    if epoch % 1000 == 999:
        print(f'Epoch: {epoch} | Train_Loss: {lossBP.item():0.5f} | Test_Loss: {test_lossBP.item():0.5f}')
    
    lossesBP.append(lossBP.item())
    accuracy_historyBP.append(accuracyBP)
    test_lossesBP.append(test_lossBP.item())
    test_accuracy_historyBP.append(test_accuracyBP)


Epoch: 999 | Train_Loss: 10.45128 | Test_Loss: 8.99051
Epoch: 1999 | Train_Loss: 10.45038 | Test_Loss: 9.03162
Epoch: 2999 | Train_Loss: 10.45037 | Test_Loss: 9.03624
Epoch: 3999 | Train_Loss: 10.45037 | Test_Loss: 9.03686
Epoch: 4999 | Train_Loss: 10.45037 | Test_Loss: 9.03693
Epoch: 5999 | Train_Loss: 10.45037 | Test_Loss: 9.03693
Epoch: 6999 | Train_Loss: 10.45037 | Test_Loss: 9.03693
Epoch: 7999 | Train_Loss: 10.45037 | Test_Loss: 9.03693
Epoch: 8999 | Train_Loss: 10.45037 | Test_Loss: 9.03693


In [8]:
print('Backpropogation')
print('weights_BP:', modelBP.state_dict()['linear.weight'].numpy())
print('bias_BP:', modelBP.state_dict()['linear.bias'].numpy())


Backpropogation
weights_BP: [[ 0.2149146   0.06529404 -0.0932102  -0.12110868]]
bias_BP: [0.6313578]


In [ ]:
#SGA
losses = []
accuracy_history = []
test_losses = []
test_accuracy_history = []

learning_rate_BP = 1e-3
learning_rate_SG = 1e-2
loss_fn = nn.MSELoss(reduction='sum')
optimizer_BP = optim.SGD(model.linear1.parameters(), lr=learning_rate_BP)
optimizer_SG = optim.SGD(model.linear2.parameters(), lr=learning_rate_SG)

for epoch in range(9000):
    model.train()
    z_pred = model(X, time)
    
    loss1 = loss_fn(z_pred, target) / 2
    loss1.backward()
    optimizer_BP.step()
    optimizer_BP.zero_grad()
    
    ########
    X1 = F.linear(X, weight = model.linear1.weight, bias = model.linear1.bias)
    X2 = F.linear(X * time.T, weight = model.linear2.weight, bias = model.linear2.bias)
    Z_pred = F.sigmoid(X1 + X2) * 2 - 1
    
    A = (Z_pred - target)
    
    B1 = F.sigmoid(X[:, :D_in] @ model.linear1.weight.T + model.linear1.bias)
    B2 = 1 - B1
    B = B1 * B2

    C1 = dotX @ model.linear1.weight.T + model.linear1.bias
    C2 = X[:, :D_in] @ model.linear2.weight.T + model.linear2.bias
    C = C1 + C2

    ABC = A * B * C
    
    dotQ_0 = 2 * ABC.sum()
    ########
    
    loss2 = loss_fn(dotQ_0, dotQmax) / 2
    loss2.backward()
    optimizer_SG.step()
    optimizer_SG.zero_grad()
    
    accuracy = (tensor(np.sign(z_pred.detach().numpy())) == target).float().mean().data.cpu()
    
    model.eval()
    test_pred = model(test, test_time)
    test_loss = loss_fn(test_pred, test_target) / 2
    test_accuracy = (tensor(np.sign(test_pred.detach().numpy())) == test_target).float().mean().data.cpu()
    
    if epoch % 1000 == 999:
        print(f'Epoch: {epoch} | Train_Loss: {loss1.item():0.5f} | Test_Loss: {test_loss.item():0.5f}')
    
    losses.append(loss1.item())
    accuracy_history.append(accuracy)
    test_losses.append(test_loss.item())
    test_accuracy_history.append(test_accuracy)


Epoch: 999 | Train_Loss: 10.17624 | Test_Loss: 12.41633
Epoch: 1999 | Train_Loss: 10.12591 | Test_Loss: 12.73812
Epoch: 2999 | Train_Loss: 10.02427 | Test_Loss: 13.55474
Epoch: 3999 | Train_Loss: 9.86384 | Test_Loss: 12.93449
Epoch: 4999 | Train_Loss: 9.70431 | Test_Loss: 11.44881
Epoch: 5999 | Train_Loss: 9.57618 | Test_Loss: 10.19990
Epoch: 6999 | Train_Loss: 9.49394 | Test_Loss: 9.46756


In [ ]:
print('SGA')
print('weights_BP:', model.state_dict()['linear1.weight'].numpy())
print('bias_BP:', model.state_dict()['linear1.bias'].numpy())
print('weights_SG:', model.state_dict()['linear2.weight'].numpy())
print('bias_SG:', model.state_dict()['linear2.bias'].numpy())


#### Results

In [ ]:
figSGA_Loss, axSGA_Loss = plt.subplots(figsize=(10, 5))
axSGA_Loss.plot(losses, '--', label='Train', color = 'black')
axSGA_Loss.plot(test_losses, '-', label='Test', color = 'black')
axSGA_Loss.set_xlabel('Epoch')
axSGA_Loss.set_ylabel('Loss')
axSGA_Loss.set_title(f"SGA Loss.  Train: {loss1.item():0.3f}  ,  Test: {test_loss.item():0.3f}")
axSGA_Loss.set_ylim([8, 18])
axSGA_Loss.grid(True)
plt.legend()
plt.savefig("SGA_Loss")

figSGA_Acc, axSGA_Acc = plt.subplots(figsize=(10, 5))
axSGA_Acc.plot(accuracy_history, '--', label='Train', color = 'black')
axSGA_Acc.plot(test_accuracy_history, '-', label='Test', color = 'black')
axSGA_Acc.set_xlabel('Epoch')
axSGA_Acc.set_ylabel('Accuracy')
axSGA_Acc.set_title(f"SGA Accuracy.  Train: {100*accuracy:0.1f}%  ,  Test: {100*test_accuracy:0.1f}%")
axSGA_Acc.set_ylim([0.3, 1])
axSGA_Acc.grid(True)
plt.legend()
plt.savefig("SGA_Accuracy")

figBP_Loss, axBP_Loss = plt.subplots(figsize=(10, 5))
axBP_Loss.plot(lossesBP, '--', label='Train', color = 'black')
axBP_Loss.plot(test_lossesBP, '-', label='Test', color = 'black')
axBP_Loss.set_xlabel('Epoch')
axBP_Loss.set_ylabel('Loss')
axBP_Loss.set_title(f"Backpropogation Loss.  Train: {lossBP.item():0.3f}  ,  Test: {test_lossBP.item():0.3f}")
axBP_Loss.set_ylim([8, 18])
axBP_Loss.grid(True)
plt.legend()
plt.savefig("BP_Loss")

figBP_Acc, axBP_Acc = plt.subplots(figsize=(10, 5))
axBP_Acc.plot(accuracy_historyBP, '--', label='Train', color = 'black')
axBP_Acc.plot(test_accuracy_historyBP, '-', label='Test', color = 'black')
axBP_Acc.set_xlabel('Epoch')
axBP_Acc.set_ylabel('Accuracy')
axBP_Acc.set_title(f"Backpropogation Accuracy.  Train: {100*accuracyBP:0.1f}%  ,  Test: {100*test_accuracyBP:0.1f}%")
axBP_Acc.set_ylim([0.3, 1])
axBP_Acc.grid(True)
plt.legend()
plt.savefig("BP_Accuracy")

plt.show()


In [ ]:
print('SGA')
print(f'Minimal train Loss     :   {np.min(losses):0.3f}')
print(f'Maximal train Accuracy :   {100*np.max(accuracy_history):0.1f}%')
print(f'Minimal test Loss      :   {np.min(test_losses):0.3f}')
print(f'Maximal test Accuracy  :   {100*np.max(test_accuracy_history):0.1f}%')

print('\nBackpropogation')
print(f'Minimal train Loss      :   {np.min(lossesBP):0.3f}')
print(f'Maximal train Accuracy :   {100*np.max(accuracy_historyBP):0.1f}%')
print(f'Minimal test Loss       :   {np.min(test_lossesBP):0.3f}')
print(f'Maximal test Accuracy  :   {100*np.max(test_accuracy_historyBP):0.1f}%')


In [ ]:
print('Backpropogation\n')
print('z_pred:', modelBP(X).data.numpy().T,  'z_target:', target.numpy().T, sep = '\n')
print('test_pred: ', test_predBP.data.numpy().T,  'test_target: ', test_target.numpy().T, sep = '\n')

print('\nSGA\n')
print('z_pred:', model(X,time).data.numpy().T, 'z_target:', target.numpy().T, sep = '\n')
print('test_pred: ', test_pred.data.numpy().T,  'test_target: ', test_target.numpy().T, sep = '\n')


In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(14, 14))

axs[0, 0].plot(target.detach().numpy(), modelBP(X).detach().numpy(), "o",  color = 'black')
axs[0, 0].set(xlabel='target', ylabel='prediction',
       title=f"BP.  Train Loss: {lossBP.item():0.3f}  ,  Train Accuracy: {100*accuracyBP:0.1f}%")
axs[0, 0].set_xlim([-1.05, 1.05])
axs[0, 0].set_ylim([-1.05, 1.05])
axs[0, 0].grid()

axs[0, 1].plot(test_target.detach().numpy(), test_predBP.detach().numpy(), "o",  color = 'black')
axs[0, 1].set(xlabel='target', ylabel='prediction',
       title=f"BP.  Test Loss: {test_lossBP.item():0.3f}  ,  Test Accuracy: {100*test_accuracyBP:0.1f}%")
axs[0, 1].set_xlim([-1.05, 1.05])
axs[0, 1].set_ylim([-1.05, 1.05])
axs[0, 1].grid()

axs[1, 0].plot(target.detach().numpy(), model(X,time).detach().numpy(), "o",  color = 'black')
axs[1, 0].set(xlabel='target', ylabel='prediction', 
       title=f"SGA.  Train Loss: {loss1.item():0.3f}  ,  Train Accuracy: {100*accuracy:0.1f}%")
axs[1, 0].set_xlim([-1.05, 1.05])
axs[1, 0].set_ylim([-1.05, 1.05])
axs[1, 0].grid()

axs[1, 1].plot(test_target.detach().numpy(), test_pred.detach().numpy(), "o", color = 'black')
axs[1, 1].set(xlabel='target', ylabel='prediction',
       title=f"SGA.  Test Loss: {test_loss.item():0.3f}  ,  Test Accuracy: {100*test_accuracy:0.1f}%")
axs[1, 1].set_xlim([-1.05, 1.05])
axs[1, 1].set_ylim([-1.05, 1.05])
axs[1, 1].grid()

for ax in axs.flat:
    ax.label_outer()

fig.savefig("Predictions.png")
plt.show()


In [ ]:
print('Backpropogatipon\n')
reportBP = classification_report(test_target.detach().numpy(), np.sign(test_predBP.detach().numpy()),
                               target_names=['Non-churned', 'Churned'])
print(reportBP)

print('SGA\n')
report = classification_report(test_target.detach().numpy(), np.sign(test_pred.detach().numpy()),
                               target_names=['Non-churned', 'Churned'])
print(report)


In [ ]:
sns.set(font_scale=1.5)
sns.set_color_codes("muted")

plt.figure(figsize=(7, 7))
fpr, tpr, thresholds = roc_curve(test_target.detach().numpy(), test_predBP.detach().numpy(), pos_label=1)
lw = 2
plt.plot(fpr, tpr, lw=lw, label='Backpropogation ROC curve')
plt.plot([0, 1], [0, 1])
plt.xlim([0.0, 1.0])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Backpropogation ROC curve')
plt.savefig("ROC_BP.png")
plt.show()

plt.figure(figsize=(7, 7))
fpr, tpr, thresholds = roc_curve(test_target.detach().numpy(), test_pred.detach().numpy(), pos_label=1)
lw = 2
plt.plot(fpr, tpr, lw=lw, label='SGA ROC curve')
plt.plot([0, 1], [0, 1])
plt.xlim([0.0, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('SGA ROC curve')
plt.savefig("ROC.png")
plt.show()
